In [1]:
import os
from itertools import combinations

from tqdm import tqdm

from dataset import read_atis
from utils import load_config

In [2]:
subset = 'train'

data = read_atis(subset)

languages = load_config()['languages']

In [3]:
if not os.path.exists(f'data/atis_{subset}_alignment/'):
    os.mkdir(f'data/atis_{subset}_alignment/')

for language1, language2 in tqdm(combinations(languages, 2), total=len(languages) * (len(languages) - 1) // 2):
    with open(f'data/atis_{subset}_alignment/{language1}_{language2}.src', 'w') as fp:
        for idx, block in data.groupby('uuid'):
            sentence1 = block.loc[block['language'] == language1]['utterance'].values[0]
            sentence2 = block.loc[block['language'] == language2]['utterance'].values[0]

            print(sentence1, sentence2, sep=' ||| ', file=fp)

    language1, language2 = language2, language1

    with open(f'data/atis_{subset}_alignment/{language1}_{language2}.src', 'w') as fp:
        for idx, block in data.groupby('uuid'):
            sentence1 = block.loc[block['language'] == language1]['utterance'].values[0]
            sentence2 = block.loc[block['language'] == language2]['utterance'].values[0]

            print(sentence1, sentence2, sep=' ||| ', file=fp)

100%|██████████| 21/21 [01:36<00:00,  4.60s/it]


In [5]:
cmd = """
DATA_FILE=data/atis_{}_alignment/{}_{}.src
MODEL_NAME_OR_PATH=bert-base-multilingual-cased
OUTPUT_FILE=data/atis_{}_alignment/{}_{}.out

CUDA_VISIBLE_DEVICES=0 awesome-align \
    --output_file=$OUTPUT_FILE \
    --model_name_or_path=$MODEL_NAME_OR_PATH \
    --data_file=$DATA_FILE \
    --extraction 'softmax' \
    --batch_size 32 \
"""

for language1, language2 in tqdm(combinations(languages, 2), total=len(languages) * (len(languages) - 1) // 2):
    os.system(cmd.format(subset, language1, language2, subset, language1, language2))
    os.system(cmd.format(subset, language2, language1, subset, language2, language1))

100%|██████████| 21/21 [12:14<00:00, 34.99s/it]
